In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import torch
from tqdm import tqdm
from tempfile import TemporaryDirectory
import imageio

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = False
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# Import the various experiments we need for the notebook
from sva.experiments import Sine2Phase

# Import the helper functions for Gaussian Processes
from sva.models.gp import EasySingleTaskGP

# Import the value function
from sva.value import SVF

# Import ask utility for Bayesian Optimization
from sva.models.gp.bo import ask

# A simple 2d example

In [ ]:
experiment = Sine2Phase()

In [ ]:
experiment.initialize_data(n=3, protocol="random", seed=123)
extent = experiment.get_experimental_domain_mpl_extent()
X = experiment.data.X

In [ ]:
# Get the ground truth results
x = experiment.get_dense_coordinates(ppd=100)
y = experiment(x)

In [ ]:
svf = SVF()

In [ ]:
value = svf(X, experiment(X)).reshape(-1, 1)

In [ ]:
# Train a GP on the current data
gp = EasySingleTaskGP.from_default(X, value)
gp.fit_mll()
mu, var = gp.predict(x)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(2, 2), sharex=True, sharey=True)

ax = axs
ax.imshow(mu.reshape(100, 100).T, extent=extent, interpolation='nearest', origin='lower')
ax.scatter(X[:, 0], X[:, 1], color="black", s=0.5)

plt.show()

# Running the experiment (short version)

In [ ]:
svf = SVF()
experiment = Sine2Phase()
experiment.initialize_data(n=3, protocol="random", seed=123)
extent = experiment.get_experimental_domain_mpl_extent()

In [ ]:
x = experiment.get_dense_coordinates(ppd=100)
y = experiment(x)

In [ ]:
experiment.run_gp_experiment(150, svf=svf)

In [ ]:
grid = np.linspace(0, 1, 100)
b = 0.5 + 0.25 * np.sin(2.0 * np.pi * grid)

In [ ]:
gp = experiment.data.history[-1]["easy_gp"]
mu, var = gp.predict(x)
mu = mu.reshape(100, 100).T
mu -= mu.min()
mu /= mu.max()
var = var.reshape(100, 100).T
var -= var.min()
var /= var.max()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(2, 2), sharex=True, sharey=True)

ax = axs

X = experiment.data.X
im = ax.imshow(mu, extent=extent, interpolation='nearest', origin='lower')
ax.scatter(X[:, 0], X[:, 1], color="red", s=0.5, zorder=3)
ax.plot(grid, b, linestyle="--", linewidth=0.5, color="black")

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")

plt.show()

## Let's make a GIF of the mean

In [ ]:
with TemporaryDirectory() as f:

    filenames = []

    for ii, datum in tqdm(enumerate(experiment.data.history)):

        gp = datum["easy_gp"]
        mu, var = gp.predict(x)
        mu = mu.reshape(100, 100).T
        mu -= mu.min()
        mu /= mu.max()
    
        fig, axs = plt.subplots(1, 1, figsize=(3, 3), sharex=True, sharey=True)
        
        ax = axs
    
        n = ii + 3
        im = ax.imshow(mu, extent=extent, interpolation='nearest', origin='lower')
        ax.scatter(X[:n, 0], X[:n, 1], color="red", s=0.5, zorder=3)
        ax.plot(grid, b, linestyle="--", linewidth=0.5, color="black")
        
        ax.set_xticks([0, 1])
        ax.set_yticks([0, 1])
        
        ax.set_xlabel("$x_1$")
        ax.set_ylabel("$x_2$")
    
        ax.set_title(f"$N={n}$")
        
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_ticks([0, 1])
        cbar.set_label(r"$\mathcal{G}(\mathbf{x}) \approx U_i(D_N)$")

        filename = f'{f}/{ii}.png'
        filenames.append(filename)

        plt.savefig(filename)
        plt.close()
    
    frames = []
    for filename in filenames:
        frames.append(imageio.imread(filename))
        
    exportname = "sine2phase.gif"
    kwargs = { 'duration': 1.0 }
    imageio.mimsave(exportname, frames, 'GIF', **kwargs)